<a href="https://colab.research.google.com/github/somilasthana/deeplearningtutorial/blob/master/Text_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function


import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print("Length {}".format(len(text)))

Length 1115394


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print("Number of Unique Character {}".format(len(vocab)))

Number of Unique Character 65


In [0]:
# Creating a mapping from unique characters to indices

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [9]:
text[:13], text_as_int[:13]

('First Citizen', array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]))

Prediction Task 

Given a character, or a sequence of characters, what is the most probable next character? RNN predicts the next char using the previously seen elements by maintaining an internal state.

RNN is feeded input sequence for fixed length seq_length as a training data ( = X ), the target are the corresponding input sequence but one shifted (=y)

In [0]:
seq_length = 100

In [0]:
examples_per_epoch = len(text)//seq_length # 11153

In [17]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])
  

Instructions for updating:
Colocations handled automatically by placer.
F
i
r
s
t


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
  input_text  = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [28]:
dataset

<DatasetV1Adapter shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [0]:
BATCH_SIZE=64

steps_per_epoch = examples_per_epoch//BATCH_SIZE
steps_per_epoch = int(steps_per_epoch)

BUFFER_SIZE=10000

In [29]:
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset # Batch of 64 with 100 training features/chars, Batch of 64 with 100 target features/chars 

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [31]:
tf.test.is_gpu_available() # GPU is available

True

In [0]:
rnn = tf.keras.layers.CuDNNGRU

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential(
      [
          tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
          rnn(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
          tf.keras.layers.Dense(vocab_size)
      ]
  )
  
  return model

In [0]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


Training the Model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state , and the input this time step 

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss=loss)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
EPOCHS=3

In [47]:
history = model.fit(
    dataset.repeat(),
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    callbacks=[checkpoint_callback]
)

Epoch 1/3
173/174 [============================>.] - ETA: 0s - loss: 2.7134WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
174/174 [==============================] - 16s 95ms/step - loss: 2.7101
Epoch 2/3
174/174 [==============================] - 12s 68ms/step - loss: 1.9430
Epoch 3/3
174/174 [==============================] - 12s 68ms/step - loss: 1.6792


In [52]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  
  num_generate= 1000
  
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  
  text_generated=[]
  
  temperature=1.0
  
  model.reset_states()
  
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    
    predictions = predictions / temperature
    
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1, 0].numpy()
    
    input_eval = tf.expand_dims([predicted_id], 0)
    
    text_generated.append(idx2char[predicted_id])
    
  return (start_string + ''.join(text_generated))

In [57]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: that that?

First CIOCE:
My lize, any madraindaking thou hurds,
That should eved here in choss'd, you she banison:
Yearshilved will be weO:
Thou thought to see, thou night wase the use.

QUEEN ELIZABETH:
Now forweds, my lord, if your courtry,
That though the pracgoneds,
That saught for never be in him.
The not they shouth, daster, what she
May comes: he crie, not in manicarair
Ir all this hanr.

PRINCE:
Now, breaks it. peep seemed.

CLARDONEN:
I thou no,
When you show, but imp by a car.

First Mandager:
And teengan, mysch.

HARG III:
But never go?
the peech of years you wilt come against then truem her,
Are all sutchis goil.

VORMINS:
What
Thou have my brootle cousind.

ROMEO:
Why, here alone, sir,
Years not in weep to things
If stake my spirce we to,
You hate me it would the beancily? what's destines in me,
Wencelice! 'tis milt!

MENCY GO:

ShepI he should
Cowite York! look; my wrife;
There art thou heave us, truy are carn of hatin.

AUTOLYCUS:
Cit;
You nore, fovery from nless!

Advanced: Customized Training

In [0]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [0]:
optimizer=tf.train.AdamOptimizer()

In [0]:
EPOCHS = 10

In [0]:
for epochs in range(EPOCHS):
  start = time.time()
  
  hidden = model.reset_states()
  
  for (batch_n, (inp, target)) in enumerate(dataset):
    with tf.GradientTape() as tape:
      predictions=model(inp)
      loss=tf.losses.sparse_softmax_cross_entropy(target, predictions)
      
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    if batch_n% 100 == 0:
      template= 'Epoch {} Batch {} Loss {}'
      print(template.format(epochs+1, batch_n, loss))
      
      
  if (epochs+1)%5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epochs))

In [72]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [73]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Whis
near'd.
You think you grace the bastle, of the air.
Barry, made; here is my oathlines? nay, thou trustle is man instrume to sove
Nou'r bound with noldines Kate; Lucentio is conjuity:
And thou conser, had straked tale, the treach
When it will even Bursh, and I am louse,
And all the woeful to this breath of her turn.

PAULINA:
To me,
Anielting hareful and impraining the house of Lusifer.
Hereform and all obysire; i' a patrecion,
It ckress of thy friends, hast thou,
Which your consul to sister his own parties
Did no ever from me their heless.
Give with all arms and men, to ask glous!
Nor I should desarp.

Cirs, I
was bound to thy message;
Protucine look him say; but nou will, not together,
Is beet from Petit onch with death.
But, were no dief stee ovater:
Trust of good scorn,
Thy father way the rest to look, much little something breathe;
For thy conceit or death,
Besides the heading sleep not this
That ever mean that knowledge him; where the time
In name until them.

MENENIUS